In [ ]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


seasons = 10
Game = Regency(True)
clear_output()
Game.add_relationship('WM','JR', At_War=1)
Game.add_troops('WM', 'Moergen', 'Irregulars')
for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.Seasons[Game.Season]['Season']
    Game.take_domain_actions()
    Game.war_move()
    Game.clean_up()
    Game.clear_screen()

df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
print(set(df['Action']))
print()
print(set(df[df['Success?']==True]['Action']))
Game.War


7
Forgot to mark a capitalitiative  2
Forgot to mark a capitalitiative 27


In [ ]:
Game.War['Notes'][0]

In [ ]:
Game.errors